In [ ]:
from random import choice
from itertools import chain
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
N_LEN = 2 ** 3  # how long is a number?
DIALECT = ("AB", "CD")
ALPHABET = tuple(chain.from_iterable([[l + r for r in DIALECT[1]] for l in DIALECT[0]]))
DELTA = 1
ALPHABET

In [ ]:
def number(seq, dialect=("AB", "CD"), delta=1):
    l = dialect[0]
    plus = set(l[:len(l) // 2])
    n = 0
    for l, _ in seq:
        delta /= 2
        n = n + (delta if l in plus else -delta)
    return n

In [ ]:
alphabet_iter = iter(ALPHABET)
NETWORK = next(alphabet_iter)
GROUP = next(alphabet_iter)
LAYER = next(alphabet_iter)

n_id = 0
g_id = 0
l_id = 0

connections = []

def read_count(seq):
    count = None
    if len(seq) >= 8:
        count = int(round(number(seq[:8], delta=2) + 3))
        seq = seq[8:]
    return seq, count


def process_group(seq, p):
    global g_id
    seq, count = read_count(seq)
    if count is not None:
        node_id = "G{}".format(g_id)
        g_id += 1
        print("group", node_id, count)
        connections.append((p, node_id))
        for _ in range(count):
            seq = process_layer(seq, node_id)
    return seq

def process_layer(seq, p):
    global l_id
    node_id = "L{}".format(l_id)
    l_id += 1
    print("layer", node_id)
    connections.append((p, node_id))
    return seq[1:]


def process_network(seq, p=None):
    global n_id
    if seq:
        node_id = "N{}".format(n_id)
        n_id += 1
        print("network", node_id)
        command = seq[0]
        seq = seq[1:]
        seq, count = read_count(seq)
        if count is not None:
            connections.append((p, node_id))
            if command == NETWORK:
                for _ in range(count):
                    seq = process_network(seq, node_id)
            if command == GROUP:
                for _ in range(count):
                    seq = process_group(seq, node_id)
    return seq

def process_sequence(seq):
    node_id = "R"
    last_len = len(seq)
    while seq:
        seq = process_network(seq, node_id)
        if len(seq) == last_len:
            seq = None  # nothing can be done anymore


In [ ]:
test_seq = [choice(ALPHABET) for _ in range(150)]
process_sequence(test_seq)
connections

In [ ]:
from graphviz import Graph

dot = Graph(engine="neato")
dot.format = 'png'

for edge in connections:
    dot.edge(*edge)

dot.render('test')
dot
